- MRR (Monthly Recurring Revenue)
- ARR (Annual Run Rate)
- TAS (Tasa de Adquisición de Suscriptores)
- LTV (Life Time Value)
- Churn rate
- Estatus de suscripción
- Más lo que decidas incluir

In [18]:
import streamlit as st
import numpy as np
import pandas as pd

In [19]:
df = pd.read_excel('payments.xlsx',sheet_name='Hoja 1 - payments (2)',index_col=0, header=1)
df = df.reset_index().set_index('id')

In [20]:
#Converting to Datetime
df['paid_at'] = pd.to_datetime(df['paid_at'],infer_datetime_format=True)
df['failed_at'] = pd.to_datetime(df['failed_at'],infer_datetime_format=True)
df['created_at'] = pd.to_datetime(df['created_at'],infer_datetime_format=True)
df['updated_at'] = pd.to_datetime(df['updated_at'],infer_datetime_format=True)
df['deleted_at'] = pd.to_datetime(df['deleted_at'],infer_datetime_format=True)
df['required_action_at'] = pd.to_datetime(df['required_action_at'],infer_datetime_format=True)
df['refunded_at'] = pd.to_datetime(df['refunded_at'],infer_datetime_format=True)
df['attended_at'] = pd.to_datetime(df['attended_at'],infer_datetime_format=True)
df['refund_pending_at'] = pd.to_datetime(df['refund_pending_at'],infer_datetime_format=True)


In [21]:
df.head()

,payable_type,payable_id,status,paid_at,failed_at,failure_reason,amount,total,created_at,updated_at,...,refund_reason_id,refund_failed_at,refund_failed_reason,payer_type,payer_id,company_id,payment_gateway_fee_amount,attended_at,payments_failed,refund_pending_at
id,,,,,,,,,,,,,,,,,,,,,
44,App\Models\Companies\Subscription,67,paid,2021-02-26 15:36:54,NaT,NaN,9900,9139,2021-02-26 15:36:54,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,59,9,761,NaT,NaN,NaT
45,App\Models\Companies\Subscription,63,paid,2021-02-26 16:11:52,NaT,NaN,3900,3390,2021-02-26 16:11:52,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,56,9,510,NaT,NaN,NaT
39,App\Models\Companies\Subscription,58,paid,2021-02-19 12:40:08,NaT,NaN,9900,9139,2021-02-19 12:40:08,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,53,9,761,NaT,NaN,NaT
46,App\Models\Companies\Subscription,68,failed,NaT,2021-02-26 20:45:48,Tarjeta rechazada.,3900,3390,2021-02-26 20:45:48,2021-04-15 12:14:09,...,NaN,NaN,NaN,App\Models\Clients\Client,57,9,510,NaT,NaN,NaT
40,App\Models\Companies\Subscription,61,paid,2021-02-23 15:55:20,NaT,NaN,3900,3390,2021-02-23 15:55:20,2021-04-15 12:14:08,...,NaN,NaN,NaN,App\Models\Clients\Client,55,9,510,NaT,NaN,NaT


In [22]:
subscribed=df.groupby(['company_id',pd.Grouper(key='paid_at',freq='MS')])['amount'].sum()
subscribed

company_id  paid_at   
9           2021-02-01      47400
            2021-03-01     341800
            2021-04-01     813100
            2021-05-01     102100
51          2021-03-01     175000
            2021-04-01     385000
            2021-05-01     210000
54          2021-03-01      79700
            2021-04-01    1780014
            2021-05-01    1006800
Name: amount, dtype: int64

In [38]:
#Monthly Recurrent Revenue

MRR = subscribed.groupby('paid_at').sum().reset_index().rename(columns={'amount':'MRR','paid_at':'Date'})
MRR

,Date,MRR
0,2021-02-01,47400
1,2021-03-01,596500
2,2021-04-01,2978114
3,2021-05-01,1318900


In [24]:
# Annual Run Rate
MRR['ARR'] = MRR['MRR']*12


In [25]:
# Average Revenue per User
ARPU = subscribed.groupby('paid_at').mean().reset_index().rename(columns={'amount':'ARPU','paid_at':'Date'})
ARPU['ARPU'] = ARPU['ARPU'].astype(int)


In [26]:
#Churn Rate
# Total # of Customers that Cancel / Total # of Customers (at given Month)

In [27]:
tot_custM =  df.groupby(['company_id','paid_at'])['status'].count().reset_index()
tot_custM = tot_custM.resample('MS',on='paid_at')['status'].count().reset_index().rename(columns={'status':'Total_users','paid_at':'Date'})
 


In [28]:
fail_custM = df[df['status']== 'failed']
fail_custM =  fail_custM.groupby(['company_id','failed_at'])['status'].count().reset_index()
fail_custM = fail_custM.resample('MS',on='failed_at')['status'].count().reset_index().rename(columns={'status':'Canceled','failed_at':'Date'})
fail_custM['Canceled'] = np.floor(fail_custM['Canceled']/4)
# fail_custM = fail_custM[fail_custM['Canceled']>=1]
canceled = fail_custM
canceled['Canceled']= fail_custM['Canceled'].astype(int)


In [29]:
churn = canceled.merge(tot_custM, how='inner', on='Date')
churn['Churn_rate'] = churn.Canceled/churn.Total_users
churn['Months'] = 1/churn.Churn_rate


In [30]:
#CLV Customer Life time Value
#ARPU*churn(months)
CLV = ARPU.merge(churn,how='inner',on='Date')
CLV['CLV'] = CLV['ARPU']*CLV['Months']
CLV['CLV'] = CLV['CLV'].astype(int,errors='ignore')


In [31]:
#CAC (TAS)
#Customer Aquisition Cost


In [32]:
SAAS = CLV
SAAS['MRR'] = MRR['MRR']
SAAS['ARR'] = MRR['ARR']

SAAS

,Date,ARPU,Canceled,Total_users,Churn_rate,Months,CLV,MRR,ARR
0,2021-02-01,47400,0,6,0.000000,inf,inf,47400,568800
1,2021-03-01,198833,2,30,0.066667,15.0,2982495.0,596500,7158000
2,2021-04-01,992704,4,92,0.043478,23.0,22832192.0,2978114,35737368
3,2021-05-01,439633,2,47,0.042553,23.5,10331375.5,1318900,15826800


In [33]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.express as px

In [34]:
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "SaaS Analytics: Understand Your Startup!"

In [35]:
fig1 = make_subplots(rows=2, cols=2, start_cell="top-left",subplot_titles=("Monthly Recurring Revenue", "Annual Run Rate", "Average Revenue per User / Life Time Value","ChurnRate / Customers"),specs=[[{"secondary_y": False}, {"secondary_y": False}],
                           [{"secondary_y": True}, {"secondary_y": True}]])      
fig1.add_trace(
    go.Bar(x=SAAS["Date"], y=SAAS["MRR"],name="MRR"),
    row=1, col=1
    )
    
fig1.add_trace(
    go.Bar(x=SAAS["Date"], y=SAAS["ARR"],name="ARR"),
    row=1, col=2
    )
fig1.add_trace(
    go.Scatter(x=SAAS["Date"], y=SAAS["ARPU"],name="ARPU"),
    row=2, col=1
    )
fig1.add_trace(
    go.Scatter(x=SAAS["Date"], y=SAAS["CLV"],name="CLTV",line=dict(dash='dash')),
    row=2, col=1,secondary_y=True
    )
# Set y-axes titles
fig1.update_yaxes(row=2, col=1,
        title_text="ARPU", 
        secondary_y=False)
fig1.update_yaxes(row=2, col=1,
        title_text="CLTV", 
        secondary_y=True)

fig1.add_trace(
    go.Scatter(x=SAAS["Date"], y=SAAS["Churn_rate"],name="Churn_rate" ,line=dict(dash='dash')),
    row=2, col=2)
fig1.add_trace(
    go.Scatter(x=SAAS["Date"], y=SAAS["Total_users"],name="Users"),
    row=2, col=2,secondary_y=True)
fig1.update_yaxes(row=2, col=2,
        title_text="Churn_Rate", 
        secondary_y=False)
fig1.update_yaxes(row=2, col=2,
        title_text="Users", 
        secondary_y=True)
fig1.update_layout(yaxis5=dict(tickformat=",.0%"))

fig1.update_layout(height=900, width=1100, title_text="Metrics",showlegend=False)

In [36]:
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="📈", className="header-emoji"),
                html.H1(
                    children="SaaS Analytics", className="header-title"
                ),
                html.P(
                    children="Understand your Startup",
                    className="header-description",
                ),
            ],
            className="header",
        ),
html.Div(
    children=[
        html.Div(
            children=dcc.Graph(
                id="chart",
                config={"displayModeBar": False},
                figure=fig1
            ),
            className="card",
        ),
    ],
    className="wrapper",
    ),
    ]
)

In [37]:

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/



2021-05-19 13:09:29.203 INFO    __main__: Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
